# Traffic sign detection and classification

In [60]:
from xml.dom import minidom
from os import walk
import pandas as pd
import cv2 as cv
import numpy as np


In [61]:
filenames = next(walk("res/annotations"), (None, None, []))[2]  # [] if no file

signs = []


for annotation in filenames:
    # parse an xml file by name
    file = minidom.parse("res/annotations/" + annotation)

    #use getElementsByTagName() to get tag
    path = "res/images/" + file.getElementsByTagName('filename')[0].firstChild.data
    filename = file.getElementsByTagName('filename')[0].firstChild.data
    name = file.getElementsByTagName('name')[0].firstChild.data
    # truncated = file.getElementsByTagName('truncated')[0].firstChild.data
    # occluded = file.getElementsByTagName('occluded')[0].firstChild.data
    # difficult = file.getElementsByTagName('difficult')[0].firstChild.data

    if name == "trafficlight" :
        continue

    signs.append([filename, name, path])

df = pd.DataFrame(signs, columns =['filename', 'name', 'path'])

In [62]:
def condition_classes(s):
    if s['name'] == 'speedlimit':
        return 0
    elif s["name"] == 'crosswalk':
        return 1
    elif s["name"] == "stop":
        return 2

df["class"] = df.apply(condition_classes, axis=1)

## Step 1 - Histogram equalization

In [71]:
def apply_histogram_equalization(row):
    img = cv.imread(row.path)
    # convert image from RGB to HSV
    img_hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
    # Histogram equalisation on the S-channel
    img_hsv[:, :, 1] = cv.equalizeHist(img_hsv[:, :, 1])
    # convert image back from HSV to RGB
    out = cv.cvtColor(img_hsv, cv.COLOR_HSV2RGB)
    vis = np.concatenate((img, out), axis=1)

    cv.imwrite("output/histogram/" + row.filename, vis)

In [72]:
df.apply(apply_histogram_equalization, axis=1);